In [2]:
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/{}/{}'
import requests
headers = {
    'Accept': 'application/json',
    'token': 'ItZCKLAcQgyDWSAapgKlybUZGZbqGVNx'
}
import pandas as pd

In [3]:
datasets = requests.get(url.format('datasets', '?limit=1000'), headers = headers)

In [4]:
datasets = pd.DataFrame(datasets.json()['results'])

In [5]:
datasets.iloc[1]

datacoverage                              1
id                                     GSOM
maxdate                          2019-04-01
mindate                          1763-01-01
name            Global Summary of the Month
uid                    gov.noaa.ncdc:C00946
Name: 1, dtype: object

In [7]:
dataCate = requests.get(url.format('datacategories', '?datasetid=GSOM&limit=1000'), headers = headers)

In [8]:
dataCate = pd.DataFrame(dataCate.json()['results'])

In [9]:
dataCate

,id,name
0,COMP,Computed
1,EVAP,Evaporation
2,LAND,Land
3,PRCP,Precipitation
4,SUN,Sunshine
5,TEMP,Air Temperature
6,WIND,Wind


In [77]:
dataType = requests.get(url.format('datatypes', \
                                   '?datasetid=GSOM&datacategoryid=WIND&limit=1000'), headers = headers)




In [78]:
dataType = pd.DataFrame(dataType.json()['results'])

In [61]:
location_cate = requests.get(url.format('locationcategories', ''), headers = headers)

location_cate = pd.DataFrame(location_cate.json()['results'])

In [62]:
location_cate

,id,name
0,CITY,City
1,CLIM_DIV,Climate Division
2,CLIM_REG,Climate Region
3,CNTRY,Country
4,CNTY,County
5,HYD_ACC,Hydrologic Accounting Unit
6,HYD_CAT,Hydrologic Cataloging Unit
7,HYD_REG,Hydrologic Region
8,HYD_SUB,Hydrologic Subregion
9,ST,State


In [6]:
location = requests.get(url.format('locations', '?datasetid=GSOM&locationcategoryid=ST&limit=1000'), headers = headers)

location = pd.DataFrame(location.json()['results'])

In [7]:
location.loc[location['name'].str.contains('Dakota')]

,datacoverage,id,maxdate,mindate,name
34,1,FIPS:38,2019-04-01,1893-01-01,North Dakota
41,1,FIPS:46,2019-04-01,1893-01-01,South Dakota


The necessary dataTypeID:
for precipitation:PRCP
for sun:TSUN
for TEMP: TAVG
for wind: AWND


Western Corn States:
Lowa IA  FIPS:19
Nebraska NE FIPS:31
Minnesota    MN FIPS:27
Kansas    KS FIPS:20
South Dakota    SD FIPS:46
North Dakota    ND FIPS:38

In [171]:
stations_prcp = requests.get(url.format('stations', \
                                       '?datasetid=GSOM&locationid={}&datatypeid=PRCP&startdate={}-01-01&enddate={}-12-31&limit=1000'\
                                            .format('FIPS:19', 1999, 2008)), headers = headers)
                                                    
                                                    

In [28]:
location_id = {
    'illinois' : 'FIPS:17',
    'ohio' : 'FIPS:39',
    'indiana' : 'FIPS:18',
    'wisconsin' : 'FIPS:55',
    'missouri' : 'FIPS:29',
    'michigan' : 'FIPS:26',
    'oklahoma' : 'FIPS:40',
    'texas' : 'FIPS:48',
    'north carolina' : 'FIPS:37',
    'iowa' : 'FIPS:19',
    'nebraska' : 'FIPS:31',
    'minnesota' : 'FIPS:27',
    'kansas' : 'FIPS:20',
    'south dakota' : 'FIPS:46',
    'north dakota' : 'FIPS:38' 
}
def data_generator(sty, edy, state):
    #The prcp data
    stations_prcp = requests.get(url.format('stations', \
                                       '?datasetid=GSOM&locationid={}&datatypeid=PRCP&startdate={}-01-01&enddate={}-12-31&limit=1000'\
                                            .format(location_id[state], sty, edy)), headers = headers)

    stations_prcp = pd.DataFrame(stations_prcp.json()['results'])

    final_res_prcp = pd.DataFrame([])
    for item in list(stations_prcp['id']):
        item = item[6:]
        try:
            mytest = pd.read_csv('/Users/yuying/cropPlus/gsom-latest/{}.csv'.format(item))
            mytest = mytest[['DATE', 'PRCP']]
        except:
            print('{} skip one station {}'.format(state, item))
            continue
        final_res_prcp = pd.concat([mytest, final_res_prcp], sort = False)
    final_res_prcp['DATE'] = pd.to_datetime(final_res_prcp['DATE'], format = '%Y-%m')
    final_res_prcp = final_res_prcp.loc[(final_res_prcp['DATE'] > '{}-12-31'.\
                                         format(sty-1))&(final_res_prcp['DATE'] < '{}-01-01'.format(edy+1))]
    final_res_prcp.dropna(inplace = True)
    final_res_prcp = final_res_prcp.groupby('DATE').mean()
    final_res_prcp.reset_index(drop = True, inplace = True)
    #The temp data
    stations_TAVG = requests.get(url.format('stations', \
                                       '?datasetid=GSOM&locationid={}&datatypeid=TAVG&startdate={}-01-01&enddate={}-12-31&limit=1000'.\
                                            format(location_id[state], sty, edy)), headers = headers)

    stations_TAVG = pd.DataFrame(stations_TAVG.json()['results'])
    final_res_tavg = pd.DataFrame([])
    for item in list(stations_TAVG['id']):
        item = item[6:]
        try:
            mytest = pd.read_csv('/Users/yuying/cropPlus/gsom-latest/{}.csv'.format(item))
            mytest = mytest[['DATE', 'TAVG']]
        except:
            print('{} skip one station {}'.format(state, item))
            continue
        final_res_tavg = pd.concat([mytest, final_res_tavg], sort = False)

    final_res_tavg = final_res_tavg.loc[(final_res_tavg['DATE'] > '{}-12'.\
                                         format(sty-1))&(final_res_tavg['DATE'] < '{}-01'.format(edy+1))]
    final_res_tavg.dropna(inplace = True)
    final_res_tavg = final_res_tavg.groupby('DATE').mean()
    final_res_tavg.reset_index(drop = True, inplace = True)


    #The awnd data
    stations_AWND = requests.get(url.format('stations', \
                                       '?datasetid=GSOM&locationid={}&datatypeid=AWND&startdate={}-01-01&enddate={}-12-31&limit=1000'.\
                                           format(location_id[state], sty, edy)), headers = headers)

    stations_AWND = pd.DataFrame(stations_AWND.json()['results'])
    final_res_awnd = pd.DataFrame([])
    for item in list(stations_AWND['id']):
        item = item[6:]
        try:
            mytest = pd.read_csv('/Users/yuying/cropPlus/gsom-latest/{}.csv'.format(item))
            mytest = mytest[['DATE', 'AWND']]
        except:
            print('{} skip one station {}'.format(state, item))
            continue
        final_res_awnd = pd.concat([mytest, final_res_awnd], sort = False)

    final_res_awnd = final_res_awnd.loc[(final_res_awnd['DATE'] > '{}-12'.\
                                        format(sty-1))&(final_res_awnd['DATE'] < '{}-01'.format(edy+1))]
    final_res_awnd.dropna(inplace = True)
    final_res_awnd = final_res_awnd.groupby('DATE').mean()
    final_res_awnd.reset_index(drop = True, inplace = True)
    final_res_awnd['AWND'] = final_res_awnd['AWND'].fillna(0)


    final_res = pd.concat([final_res_prcp, final_res_tavg, final_res_awnd], axis = 1)

    daterange1 = pd.date_range('{}-01'.format(sty), '{}-01'.format(edy+1), freq='M')
    daterange1 = list(daterange1.strftime('%Y-%m-%d'))

    final_res['DATE'] = daterange1
    final_res['DATE'] = final_res['DATE'].apply(str)

    final_res_diff = pd.DataFrame([])
    for i in range(1, 13, 1):
        if i < 10:
             i = '0' + str(i)
        else:
            i = str(i)
        temp = final_res.loc[final_res['DATE'].str.contains('-{}-'.format(i))]
        temp_slice = temp[['PRCP', 'TAVG', 'AWND']]

        temp_res = temp_slice - temp_slice.iloc[0]

        temp_res['DATE'] = temp['DATE']

        final_res_diff = pd.concat([temp_res, final_res_diff], sort = False)

    new_final = pd.DataFrame(list(range(sty, edy+1)), columns = ['DATE'])

    for yr in list(range(sty, edy+1)):
        for i in range(1, 13, 1):
            if i < 10:
                 i = '0' + str(i)
            else:
                i = str(i)
            my_temp1 = final_res_diff.loc[final_res_diff['DATE'].str.contains('{}-{}-'.format(yr, i))]['PRCP']
            new_final.loc[new_final['DATE'] == yr, 'delta_prcp_{}'.format(i)] = round(float(my_temp1), 2)
            my_temp2 = final_res_diff.loc[final_res_diff['DATE'].str.contains('{}-{}-'.format(yr, i))]['TAVG']
            new_final.loc[new_final['DATE'] == yr, 'delta_tavg_{}'.format(i)] = round(float(my_temp2), 2)
            my_temp3 = final_res_diff.loc[final_res_diff['DATE'].str.contains('{}-{}-'.format(yr, i))]['AWND']
            new_final.loc[new_final['DATE'] == yr, 'delta_awnd_{}'.format(i)] = round(float(my_temp3), 2)
    y1 = pd.read_csv('corn_yield_{}_1999_2008.csv'.format(state))
    y2 = pd.read_csv('corn_yield_{}_2009_2018.csv'.format(state))
    corn_yield = pd.concat([y1, y2], sort = 'False')
    corn_yield = corn_yield.loc[(corn_yield['Year']>=sty)&(corn_yield['Year']<=edy)]
    corn_yield = corn_yield[['Year', 'Value']]

    corn_yield.sort_values(by = ['Year'], inplace = True)

    corn_yield['Value'] = corn_yield['Value'] - corn_yield['Value'].iloc[0]

    new_final = new_final.merge(corn_yield, left_on = 'DATE', right_on = 'Year')

    new_final.drop(axis = 1, columns = ['Year'], inplace = True)
    new_final.rename(columns = {'Value' : 'delta_yield_corn_{}'.format(state)}, inplace = True)
    new_final.drop(axis = 1, columns = \
                      ['delta_prcp_11', 'delta_tavg_11','delta_awnd_11', \
                       'delta_prcp_12', 'delta_tavg_12', 'delta_awnd_12'], inplace = True)
    return new_final
    #new_final.to_csv('final_data_combined_yield_{}_{}_{}.csv'.format(state, sty, edy), index = False)

In [7]:
states = list(location_id.keys())

In [11]:
nw = ['north dakota']
w = ['south dakota', 'iowa', 'nebraska', 'kansas']
e = ['ohio', 'missouri', 'indiana', 'illinois']
sw = ['oklahoma', 'texas']
ne = ['michigan', 'minnesota', 'wisconsin']
se = ['north carolina']

In [12]:
regions = {'w' : w, 'sw' : sw, 'nw' : nw, 'ne' : ne, 'e' : e}

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
def back_test(area, sty, edy, alpha):
    #get training set
    ds = pd.DataFrame([])
    for state in area:
        temp = data_generator(sty, edy, state)
        ds = pd.concat([temp, ds], sort = False)
    ds = ds.loc[ds['DATE'] != sty].copy()
    ds.fillna(0, inplace = True)
    ds['value'] = ds.iloc[:, -len(area):].sum(axis = 1)
    scaler = StandardScaler()
    X = scaler.fit_transform(ds.iloc[:, 1:31].values)
    clf = linear_model.Lasso(alpha = alpha, fit_intercept = True)
    clf.fit(X, ds['value'].values)
    #get test set
    ts = pd.DataFrame([])
    for state in area:
        temp2 = data_generator(sty, edy+1, state)
        ts = pd.concat([temp2, ts], sort = False)
    ts.fillna(0, inplace = True)
    ts['value'] = ts.iloc[:, -len(area):].sum(axis = 1)
    testX = scaler.fit_transform(ts.iloc[:, 1:31].values)
    ts['pred'] = clf.predict(testX)
    ts = ts.loc[ts['DATE'] == edy+1].copy()
    return ts

In [30]:
#back test
for area in list(regions.keys()):
    for a in [0.8, 1]:
        final_res = pd.DataFrame([])
        for i in range(9):
            temp = back_test(regions[area], 1999+i, 1999+10+i, alpha = a)
            final_res = pd.concat([temp, final_res], sort = False)
        final_res.to_csv('back_test/{}/corn_back_test_alpha{}.csv'.format(area, a), index = False)

nebraska skip one station US10arth001
nebraska skip one station US10chey001
nebraska skip one station US10furn011
nebraska skip one station US10jeff017
nebraska skip one station US10rich004
nebraska skip one station US10sali027
kansas skip one station US1KSCL0002
kansas skip one station US1KSCY0004
kansas skip one station US1KSWB0003
nebraska skip one station US10arth001
nebraska skip one station US10chey001
nebraska skip one station US10furn011
nebraska skip one station US10jeff017
nebraska skip one station US10rich004
nebraska skip one station US10sali027
kansas skip one station US1KSCL0002
kansas skip one station US1KSCY0004
kansas skip one station US1KSWB0003
nebraska skip one station US10arth001
nebraska skip one station US10chey001
nebraska skip one station US10furn011
nebraska skip one station US10jeff017
nebraska skip one station US10rich004
nebraska skip one station US10sali027
kansas skip one station US1KSCL0002
kansas skip one station US1KSCY0004
kansas skip one station US1K

kansas skip one station US1KSLV0010
kansas skip one station US1KSMN0025
kansas skip one station US1KSMS0005
kansas skip one station US1KSNM0006
kansas skip one station US1KSOS0023
kansas skip one station US1KSPR0018
kansas skip one station US1KSRA0019
kansas skip one station US1KSRA0025
kansas skip one station US1KSRH0015
kansas skip one station US1KSRL0048
kansas skip one station US1KSRO0010
kansas skip one station US1KSSA0029
kansas skip one station US1KSWB0003
kansas skip one station US1KSWH0010
kansas skip one station USC00140497
kansas skip one station USC00140497
nebraska skip one station US10arth001
nebraska skip one station US10chey001
nebraska skip one station US10furn011
nebraska skip one station US10jeff017
nebraska skip one station US10rich004
nebraska skip one station US10sali027
kansas skip one station US1KSCL0002
kansas skip one station US1KSCY0004
kansas skip one station US1KSWB0003
nebraska skip one station US10arth001
nebraska skip one station US10chey001
nebraska ski

nebraska skip one station US10chey001
nebraska skip one station US10furn011
nebraska skip one station US10jeff017
nebraska skip one station US10rich004
nebraska skip one station US10sali027
kansas skip one station US1KSBR0002
kansas skip one station US1KSBR0004
kansas skip one station US1KSBR0005
kansas skip one station US1KSBU0034
kansas skip one station US1KSCL0002
kansas skip one station US1KSCY0004
kansas skip one station US1KSCY0015
kansas skip one station US1KSDG0063
kansas skip one station US1KSDG0064
kansas skip one station US1KSDP0016
kansas skip one station US1KSEL0095
kansas skip one station US1KSLV0010
kansas skip one station US1KSMN0025
kansas skip one station US1KSMS0005
kansas skip one station US1KSNM0006
kansas skip one station US1KSOS0023
kansas skip one station US1KSPR0018
kansas skip one station US1KSRA0019
kansas skip one station US1KSRA0025
kansas skip one station US1KSRH0015
kansas skip one station US1KSRL0048
kansas skip one station US1KSRO0010
kansas skip one st

north dakota skip one station US1NDMT0002
north dakota skip one station US1NDBH0004
north dakota skip one station US1NDDN0002
north dakota skip one station US1NDMT0002
north dakota skip one station US1NDBH0004
north dakota skip one station US1NDDN0002
north dakota skip one station US1NDMT0002
north dakota skip one station US1NDBH0004
north dakota skip one station US1NDDN0002
north dakota skip one station US1NDMT0002
north dakota skip one station US1NDBH0004
north dakota skip one station US1NDDN0002
north dakota skip one station US1NDMT0002
north dakota skip one station US1NDBH0004
north dakota skip one station US1NDDN0002
north dakota skip one station US1NDMT0002
north dakota skip one station US1NDBH0004
north dakota skip one station US1NDDN0002
north dakota skip one station US1NDMT0002
north dakota skip one station US1NDBH0004
north dakota skip one station US1NDDN0002
north dakota skip one station US1NDMT0002
north dakota skip one station US1NDBH0004
north dakota skip one station US1N

minnesota skip one station US1MNHN0005
wisconsin skip one station US1WIBY0002
michigan skip one station US1MIDL0001
michigan skip one station US1MIHG0011
michigan skip one station US1MIHG0018
michigan skip one station US1MIKZ0006
michigan skip one station US1MIMC0004
michigan skip one station US1MIMR0001
michigan skip one station US1MIOC0003
michigan skip one station US1MIWY0007
minnesota skip one station US1MNHN0005
wisconsin skip one station US1WIBY0002
michigan skip one station US1MICS0010
michigan skip one station US1MICT0018
michigan skip one station US1MIDL0001
michigan skip one station US1MIHG0011
michigan skip one station US1MIHG0018
michigan skip one station US1MIKZ0006
michigan skip one station US1MIMC0004
michigan skip one station US1MIMR0001
michigan skip one station US1MIOC0003
michigan skip one station US1MIOW0049
michigan skip one station US1MIWY0007
michigan skip one station USC00204105
michigan skip one station USC00208245
michigan skip one station USC00204105
minnesot

missouri skip one station US1MOBN0011
missouri skip one station US1MOBT0001
missouri skip one station US1MOCS0002
missouri skip one station US1MOLN0001
missouri skip one station US1MOWH0001
indiana skip one station US1INBN0035
indiana skip one station US1INHY0007
indiana skip one station US1INMR0003
indiana skip one station US1INTN0001
illinois skip one station US1ILCP0064
illinois skip one station US1ILDG0004
illinois skip one station US1ILKN0030
illinois skip one station US1ILLV0007
illinois skip one station US1ILWN0002
missouri skip one station US1MOBN0011
missouri skip one station US1MOBT0001
missouri skip one station US1MOCS0002
missouri skip one station US1MOLN0001
missouri skip one station US1MOWH0001
indiana skip one station US1INBN0035
indiana skip one station US1INHY0007
indiana skip one station US1INMR0003
indiana skip one station US1INTN0001
illinois skip one station US1ILCP0064
illinois skip one station US1ILDG0004
illinois skip one station US1ILKN0030
illinois skip one st

indiana skip one station US1INMR0003
indiana skip one station US1INTN0001
illinois skip one station US1ILBU0005
illinois skip one station US1ILCP0064
illinois skip one station US1ILDG0004
illinois skip one station US1ILKN0030
illinois skip one station US1ILLV0007
illinois skip one station US1ILME0008
illinois skip one station US1ILWN0002
missouri skip one station US1MOBN0011
missouri skip one station US1MOBT0001
missouri skip one station US1MOCS0002
missouri skip one station US1MOLN0001
missouri skip one station US1MOWH0001
indiana skip one station US1INBN0035
indiana skip one station US1INHY0007
indiana skip one station US1INHY0025
indiana skip one station US1INMR0003
indiana skip one station US1INTN0001
illinois skip one station US1ILBU0005
illinois skip one station US1ILCP0064
illinois skip one station US1ILDG0004
illinois skip one station US1ILKN0030
illinois skip one station US1ILLV0007
illinois skip one station US1ILME0008
missouri skip one station US1MOBN0011
missouri skip one s

indiana skip one station US1INTN0001
illinois skip one station US1ILBU0005
illinois skip one station US1ILCP0064
illinois skip one station US1ILDG0004
illinois skip one station US1ILKN0030
illinois skip one station US1ILLV0007
illinois skip one station US1ILWN0002
missouri skip one station US1MOBN0011
missouri skip one station US1MOBT0001
missouri skip one station US1MOCS0002
missouri skip one station US1MOLN0001
missouri skip one station US1MOWH0001
indiana skip one station US1INBN0035
indiana skip one station US1INHY0007
indiana skip one station US1INMR0003
indiana skip one station US1INTN0001
illinois skip one station US1ILBU0005
illinois skip one station US1ILCP0064
illinois skip one station US1ILDG0004
illinois skip one station US1ILKN0030
illinois skip one station US1ILLV0007
illinois skip one station US1ILME0008
illinois skip one station US1ILWN0002
missouri skip one station US1MOBN0011
missouri skip one station US1MOBT0001
missouri skip one station US1MOCS0002
missouri skip one

In [19]:
corn_yield = pd.read_csv('corn_yield_iowa_1999_2008.csv')

In [56]:
list(corn_yield.index)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [57]:
# add back absolute value for yield and prediction
for area in list(regions.keys()):
    final_res = pd.read_csv('back_test/{}/corn_back_test_alpha0.8.csv'.format(area))
    for state in regions[area]:
        corn_yield = pd.read_csv('corn_yield_{}_1999_2008.csv'.format(state))
        check = final_res.loc[final_res['delta_yield_corn_{}'.format(state)] == 0].drop
        for i in range(9):
            benchmark = 1999+i
            predict = 1999+10+i+1
            bench_yield = corn_yield.loc[corn_yield['Year'] == benchmark, 'Value']
            prior = final_res.loc[(final_res['DATE']==predict)&(final_res['delta_yield_corn_{}'.format(state)] \
                                                                == final_res['value'])]
            if prior.shape[0] > 1:
                prior = prior.loc[prior['delta_yield_corn_{}'.format(state)] != 0]
            index = list(prior.index)[0]
            prior.reset_index(drop = True, inplace = True)
            prior = prior.loc[0, 'value']
            final_res.loc[index, 'value'] += float(bench_yield)
            final_res.loc[index, 'pred'] += float(bench_yield)
    final_res.to_csv('back_test/{}/corn_back_test_alpha0.8_with_abs_yield.csv'.format(area), index = False)
            
        

In [63]:
#plot prediction result
import plotly.graph_objs as go
import plotly
for area in regions.keys():
    final_res = pd.read_csv('back_test/{}/corn_back_test_alpha0.8_with_abs_yield.csv'.format(area))
    X_axis = list(final_res['DATE'].drop_duplicates())
    for state in regions[area]:
        temp = pd.DataFrame([])
        for i in range(9):
            predict = 1999+10+i+1
            t1 = final_res.loc[(final_res['DATE'] == predict)&(final_res['delta_yield_corn_{}'.format(state)] != 0)]
            if t1.empty:
                t1 = final_res.loc[(final_res['DATE'] == predict)&(final_res['delta_yield_corn_{}'.format(state)] == 0)]
            temp = pd.concat([t1, temp], sort = False)
        y0 = temp['value']
        y1 = temp['pred']
        trace0 = go.Scatter(x = X_axis, y = y0, name = '{} corn actual yield'.format(state))
        trace1 = go.Scatter(x = X_axis, y = y1, name = '{} corn predicted yield'.format(state))
        data = [trace0, trace1]
        layout = dict(title = 'corn yield backtest(BU/ACRE)', xaxis = dict(title = 'Years'), \
                      yaxis = dict(title = 'yield(BU/ACRE)'))
        fig = dict(data = data, layout = layout)
        plotly.offline.plot(fig, filename = 'back_test/{}/{} yield prediction.html'.format(area, state),\
                            auto_open = False)